# DATA SCIENCE II

*PRIMER PRE-ENTREGA 17/10/2025 - AGOSTINA FONTANA*

Para el trabajo de Data Science II voy a tomar como base, el Dataset utilizado en la entrega final de Data Science I, donde indagué sobre el mercado inmobiliario, tomando como dataset un reporte de Kaggle respecto a las propiedades públicadas en la página web de "Properati".

A continuación, detallo el link de Github donde se encuentra publicado el trabajo efectuado en el curso anterior: 
https://github.com/AgostinaFontana/ProyectoDS/blob/main/ProyectoParteIII%2BFontana.ipynb 

La temática seleccionada surge a partir del interés en efectuar una comparativa de los precios de las propiedades basadas en diferentes atributos en el norte de Gran Buenos Aires.

Con la apertura de la línea de crédito hipotecario en 2024, una de las hipótesis que quise probar en el trabajo del curso anterior fue comparar los valores de los alquileres vs los valores de pago de una cuota de crédito. 

Al tener muchos valores de alquiler en pesos argentinos tuve una limitación a mi análisis, no pudiendo comprobar o refutar la hipotesis inicialmente planteada dado que las propiedades no se encontraban en una moneda homogénea. 



Durante esta primer entrega estaremos enfocándonos en los siguientes puntos: 
1) DATA ACQUISITION: Descarga de datos de APIs Públicas.
 Estaremos indagando una API que nos permita traernos el tipo de cambio USD-ARS con el fin de poder dolarizar toda la base de propiedades tanto para operaciones de venta como de alquiler 
2) DATA WRANGLING: Limpieza de datos. Eliminación de datos duplicados y tratamiento de trabajos nulos.
Estaremos realizando un trabajo similar al efectuado en DATA I y estaremos aplicando técnicas adicionales en caso de corresponder con la suma de datos obtenido de la API. 
3) DATA STORYTELLING: Narrativa y gráficos.
Nos permitirán dar respuesta a preguntas / hipotésis para nuestro proyecto final 

In [51]:
# Importación de librerias

# Para exploración y manipulación de datos
import pandas as pd
import numpy as np

# Para generar herramientas de visualización
import matplotlib as plt
import seaborn as sns

## 0. IMPORTACION DEL DATASET

In [52]:
# Importación del dataset
# Para acceder al dataset
import kagglehub as kh 
# Descargo el dataset seleccionado
path = kh.dataset_download("alejandroczernikier/properati-argentina-dataset")
# Verifico la ruta descargada
print ('Ruta del dataset',path)

Ruta del dataset C:\Users\afontan1\.cache\kagglehub\datasets\alejandroczernikier\properati-argentina-dataset\versions\1


In [53]:
# Valido si dentro de la carpeta tengo archivos y si se encuentra con extensión CSV. 
# La librería "os" nos va a permitir interactuar con el sistema operativo y validar si el archivo existe
import os
Archivos_CSV= [f for f in os.listdir(path) if f.endswith('.csv')]
print ('Archivos CSV Encontrados',Archivos_CSV)

Archivos CSV Encontrados ['entrenamiento.csv']


In [54]:
# Defino el Data Frame leyendo el CSV que se encuentra en la ruta del data set obtenida en el paso anterior
df= pd.read_csv('C:/Users/afontan1/.cache/kagglehub/datasets/alejandroczernikier/properati-argentina-dataset/versions/1/entrenamiento.csv')

### ANÁLISIS EXPLORATORIO DE DATOS - DATAFRAME INICIAL (EDA) 

In [55]:
# Para ver la cantidad de filas y columnas que tiene mi dataset: 
df.shape

(992192, 25)

In [56]:
# Para ver las columnas y el tipo de datos que tiene mi data set
# Indico que me traiga el 100% de las columnas:
pd.set_option('display.max_columns',None)
# Indico que me traiga las columnas y las primeras 5 filas de mi dataset
df.head()

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,currency,price_period,title,description,property_type,operation_type,price
0,556713,Propiedad,2019-11-29,9999-12-31,2019-11-29,-58.442399,-34.573623,Argentina,Capital Federal,Colegiales,NaN,NaN,NaN,3.0,2.0,2.0,NaN,NaN,USD,NaN,"Departamento en Venta en Belgrano, Capital fed...","Sup total por escritura: 96,47 m2 (cubiertos: ...",Departamento,Venta,259000.0
1,192912,Propiedad,2020-06-05,2020-06-08,2020-06-05,-58.430493,-34.606620,Argentina,Capital Federal,Almagro,NaN,NaN,NaN,3.0,2.0,2.0,77.0,67.0,USD,NaN,Departamento de 3 ambientes en Venta en Almagro,Excelente departamento de tres ambientes ampli...,Departamento,Venta,235500.0
2,238224,Propiedad,2020-07-01,9999-12-31,2020-07-01,-58.491760,-34.574123,Argentina,Capital Federal,Villa Urquiza,NaN,NaN,NaN,2.0,NaN,1.0,60.0,55.0,USD,NaN,Andonaegui 2600 4° - - Departamento en Venta,Excelente 3 ambientes al frente con balcón. Vi...,Departamento,Venta,175000.0
3,257134,Propiedad,2019-08-17,9999-12-31,2019-08-17,-58.420737,-34.631770,Argentina,Capital Federal,Boedo,NaN,NaN,NaN,2.0,1.0,1.0,74.0,47.0,USD,NaN,PH Venta Boedo 2 amb Patio,Corredor Responsable: MARCELO TRUJILLO - CPI ...,PH,Venta,140000.0
4,521738,Propiedad,2019-08-05,2019-08-31,2019-08-05,-58.429983,-34.607225,Argentina,Capital Federal,Almagro,NaN,NaN,NaN,3.0,2.0,1.0,66.0,64.0,USD,NaN,Venta 3 Ambientes - Almagro - Balcón - Ameniti...,Corredor Responsable: Marcelo Trujillo - CUCIC...,Departamento,Venta,173000.0


In [57]:
# Utilizo la fórmula "describe" para poder obtener mayor información de mi dataset a través de la estadística
df.describe() 

,id,lat,lon,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price
count,992192.000000,838994.000000,839929.000000,0.0,503045.000000,390699.000000,773021.000000,448164.00000,4.369610e+05,9.478110e+05
mean,500338.438758,-59.266586,-34.793745,NaN,2.858577,2.086094,1.697637,493.51136,5.769533e+02,2.881687e+05
std,288685.497758,2.707383,3.066257,NaN,1.682454,2.705342,1.072722,4101.38179,7.383998e+04,5.560863e+06
min,1.000000,-180.000000,-54.842464,NaN,1.000000,-13.000000,1.000000,-200.00000,-1.700000e+02,0.000000e+00
25%,250259.750000,-58.756011,-34.921709,NaN,2.000000,1.000000,1.000000,50.00000,4.500000e+01,3.900000e+04
50%,500641.500000,-58.444383,-34.610508,NaN,3.000000,2.000000,1.000000,95.00000,7.500000e+01,9.800000e+04
75%,750507.250000,-58.058983,-34.459156,NaN,4.000000,3.000000,2.000000,270.00000,1.550000e+02,2.100000e+05
max,1000000.000000,-35.045025,85.051129,NaN,40.000000,900.000000,20.000000,200000.00000,2.700000e+07,3.100000e+09


In [58]:
# Utilizo info para poder ver el formato de los datos de cada columna y la cantidad de datos no nulos 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992192 entries, 0 to 992191
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               992192 non-null  int64  
 1   ad_type          992192 non-null  object 
 2   start_date       992192 non-null  object 
 3   end_date         992192 non-null  object 
 4   created_on       992192 non-null  object 
 5   lat              838994 non-null  float64
 6   lon              839929 non-null  float64
 7   l1               992192 non-null  object 
 8   l2               992192 non-null  object 
 9   l3               934737 non-null  object 
 10  l4               225395 non-null  object 
 11  l5               4711 non-null    object 
 12  l6               0 non-null       float64
 13  rooms            503045 non-null  float64
 14  bedrooms         390699 non-null  float64
 15  bathrooms        773021 non-null  float64
 16  surface_total    448164 non-null  floa

Para poder realizar el análisis de los valores de las propiedades en USD, va a ser necesario saber la fecha de publicación de la propiedad para poder tener conocimiento de la temporalidad y el TC a esa fecha o a ese mes. 
La columna que debemos tomar en cuenta es "start_date". En info me indica que es object por lo cual debo realizar un trabajo de tratamiento de ese dato para convertirlo a formato fecha. 

In [59]:
# Modifico el formato a fecha
df['start_date']=pd.to_datetime(df['start_date'])

In [60]:
# Verificamos que se haya modificado el type de la columna start_date 
df['start_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 992192 entries, 0 to 992191
Series name: start_date
Non-Null Count   Dtype         
--------------   -----         
992192 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.6 MB


In [61]:

# Agregar columna con el año
df['año_publicacion'] = pd.to_datetime(df['start_date']).dt.year

In [62]:
df.groupby('año_publicacion').size()

año_publicacion
2019    343341
2020    648851
dtype: int64

## 1. DATA ACQUISITION
 Descarga de datos desde API's Públicas

Para complementar el Dataset seleccionado en el trabajo final de Data Science I voy a explorar una API para poder obtener el tipo de cambio que me permita convertir los valores de las propiedades que se encuentran en ARS en USD y pueda tener una base homologada y comparable tanto para las operaciones de venta como para las operaciones de alquiler

Voy a acceder a las APIs públicas del Banco Central de la República Argentina (BCRA) 

In [63]:
pip install pyBCRAdata

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
from pyBCRAdata import BCRAclient


In [65]:
# Inicializar cliente
client = BCRAclient()

In [66]:
# Obtener cotización histórica del dólar (Traigo cotización 2019 y 2020 tomando como base el año obtenido anteriormente de año_publicación)
usd = client.currency.series(
    moneda="USD",
    fechadesde="2019-01-01",
    fechahasta="2020-12-31"
)


In [67]:
usd.head()

,fecha,detalle_codigoMoneda,detalle_descripcion,detalle_tipoPase,detalle_tipoCotizacion
0,2020-12-31,USD,DOLAR E.E.U.U.,0.0,84.15
1,2020-12-30,USD,DOLAR E.E.U.U.,0.0,84.15
2,2020-12-29,USD,DOLAR E.E.U.U.,0.0,84.02
3,2020-12-28,USD,DOLAR E.E.U.U.,0.0,83.90
4,2020-12-23,USD,DOLAR E.E.U.U.,0.0,83.36


In [68]:
usd['fecha']=pd.to_datetime(usd['fecha'])

In [69]:
# Extraer mes y año
df['mes'] = df['start_date'].dt.month
df['año'] = df['start_date'].dt.year

usd['mes'] = usd['fecha'].dt.month
usd['año'] = usd['fecha'].dt.year


In [70]:
#Concateno el mes y el año en el dataframe df y en el de usd
df['mes&año'] = df['mes'].astype(str) + '&' + df['año'].astype(str)

usd['mes&año'] = usd['mes'].astype(str) + '&' + usd['año'].astype(str)

In [71]:
#Agrupo por mes&año y calculo el promedio mensual

usd_mensual=usd.groupby('mes&año')['detalle_tipoCotizacion'].mean().reset_index()


In [72]:
#Renombro la columna de cotización

usd_mensual.rename(columns={'detalle_tipoCotizacion':'Cotizacion_Promedio_Mensual'},inplace=True)

In [73]:
#Merge entre df y usd_mensual usando la columna 'mes&año' como nexo
df_api=pd.merge(df,usd_mensual,on='mes&año',how='left')

In [74]:
df_api.shape

(992192, 30)

## 2. DATA WRANGLING
Como en el trabajo realizado en Data Science I nos concentramos en las propiedades con operaciones de venta del norte de Gran Buenos Aires, no pudiendo efectuar el análisis de las propiedades en alquiler por encontrarse en ARS, nos estaremos concentrando en el trabajo de Data II en analizar los valores de las propiedades en alquiler para luego poder unir los dos análisis efectuados y poder realizar una correcta comparación


### MANIPULACIÓN DE DATOS

A) SELECCIÓN / ELIMINACIÓN DE COLUMNAS: Analizaremos los datos que contienen las columnas del dataset y nos quedaremos con aquellas que nos generan valor agregado para el análisis que queremos efectuar. 

B) SELECCIÓN / ELIMINACIÓN DE FILAS: No tendremos en cuenta todas las propiedades del Dataset. Realizaremos un filtro de filas en base a: 
    
- Ubicación geográfica de acuerdo a la zona de interés.
- Tipo de propiedad
- Tipo de Operación

C) TRATAMIENTO DE DATOS NULOS: Identificaremos aquellas columnas con filas sin datos y analizaremos la estrategia optima para cada caso (eliminación de fila, eliminación de columna, imputación de valor de acuerdo a la moda, mediana o modo, etc.)

#### A) SELECCIÓN / ELIMINACIÓN DE COLUMNAS

In [75]:
# Volvemos a realizar un análisis exploratorio del dataframe

df_api.head(1)

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,currency,price_period,title,description,property_type,operation_type,price,año_publicacion,mes,año,mes&año,Cotizacion_Promedio_Mensual
0,556713,Propiedad,2019-11-29,9999-12-31,2019-11-29,-58.442399,-34.573623,Argentina,Capital Federal,Colegiales,NaN,NaN,NaN,3.0,2.0,2.0,NaN,NaN,USD,NaN,"Departamento en Venta en Belgrano, Capital fed...","Sup total por escritura: 96,47 m2 (cubiertos: ...",Departamento,Venta,259000.0,2019,11,2019,11&2019,59.745263


Voy a eliminar columnas que no me generan valor para mi análisis como:
-  ad_type: En todas las líneas indica "Propiedad". No genera valor agregado a mi análisis.
-  Title y Description: Si bien, nos puede brindar información de la propiedad específica, al ser campos de texto libre es dificultoso normalizar y encontrar un patrón o tendencia en dichos datos.
- Datos de geolocalización: l5 y l6. Si bien me brinda mayor capilaridad de información respecto a la geolocalización de la propiedad, no todas las propiedades cuentan con ese nivel de detalle.
- Datos de fecha de la publicación: Nos vamos a quedar solo con el dato de start_date, eliminando los campos end_date o created_on. Si bien son datos que nos van a permitir hasta que fecha estuvo publicada la propiedad, no es relevante para el análisis que queremos efectuar que no consiste en encontrar una propiedad determinada sino realizar una comparativa de precios de propiedades de acuerdo a determinados atributos.

In [76]:
df_analisis=df_api.drop(['ad_type','l6','l5','title','description','created_on', 'end_date'],axis=1)

In [77]:
# Verificamos que se hayan mantenido la cantidad de filas y se hayan reducido las columnas de 30 a 23 (eliminamos 7)
df_analisis.shape

(992192, 23)

#### B) SELECCION / ELIMINACIÓN DE FILAS

Para nuestro análisis vamos a realizar un filtro de las siguientes propiedades: 
* B.1: operation_type = 'Alquiler'
* B.2: property_type = 'Casa'
* B.3: l2 = 'Bs.As. G.B.A. Zona Norte'

In [78]:
# Indico los filtros que quiero realizar
df_filtros= (df_analisis['l2']=='Bs.As. G.B.A. Zona Norte') & (df_analisis['operation_type']=='Alquiler') & (df_analisis['property_type']=='Casa')

In [79]:
#Aplico los filtros al df df_analisis para generar el nuevo data frame de casas en alquiler en GBA Zona Norte
df_filtrado = df_analisis[df_filtros]
df_filtrado.shape

(10166, 23)

Vamos a analizar la cantidad de registros nulos por cada uno de los atributos del dataframe y en base a ello, analizaremos que estrategia utilizaremos para cada caso

In [80]:
# Los detectamos utilizando "isnull" sobre el dataframe y utilizamos la suma para que nos cuente los que son nulos (true=1)
df_filtrado.isnull().sum()

id                                0
start_date                        0
lat                            3082
lon                            3076
l1                                0
l2                                0
l3                              206
l4                             3193
rooms                          5921
bedrooms                       2932
bathrooms                       509
surface_total                  1969
surface_covered                1052
currency                        285
price_period                   5779
property_type                     0
operation_type                    0
price                           269
año_publicacion                   0
mes                               0
año                               0
mes&año                           0
Cotizacion_Promedio_Mensual       0
dtype: int64

#### C) TRATAMIENTO DE DATOS NULOS

De las 23 columnas, 12 atributos contienen datos nulos en las 10166 propiedades en alquiler de nuestro dataset. 

El tratamiento que realizaremos va a diferir de la cantidad de datos nulos y la naturaleza del dato y la importancia para nuestro análisis. 

A continuación detallamos la estrategia para cada caso: 
* C.1) Relleno el dato con un valor constante:
    - Lat, Long, l3 y l4: Me brindan información de la georeferencia de la propiedad.
    En caso que no tengan este dato coloco ""
* C.2) Elimino columnas de datos con baja relevancia para mi análisis
    - Price_period: Me quedo con el valor de start_date para trabajar con la temporalidad de la publicación y el valor a esa fecha.
    - Rooms: Al no contar con este dato en casi la mitad del dataset lo vamos a eliminar para todos los registros.

* C.3) Elimino filas que contienen valores nulos: 
    - Surface_Total, Surface_Covered y Price: Son datos fundamentales que me van a permitir analizar el valor por m2 de la propiedad. Analizar propiedades sin esos datos no me va a permitir realizar una comparativa adecuada.
    - Currency: Que me indiquen en que moneda se encuentran las propiedades es fundamental para saber si debo aplicarle el tipo de cambio obtenido con la API o no.

* C.4) Imputo los valores nulos con la media (medida de tendencia central):
    - Bathroom, bedrooms: Voy a imputar el valor medio de los datos, pero lo voy a imputar de acuerdo a la media de propiedades con dimensiones similares (metros cuadrados cubiertos)

In [86]:
#C.1 - Relleno el dato con un valor constante 
df_filtrado[['lat','lon','l3','l4']]= df_filtrado[['lat','lon','l3','l4']].fillna(0)

C:\Users\afontan1\AppData\Local\Temp\ipykernel_11296\133297313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[['lat','lon','l3','l4']]= df_filtrado[['lat','lon','l3','l4']].fillna(0)


Voy a renombrar el DF debido a que los próximos pasos consisten en eliminar filas o columnas y no quiero afectar el df_analisis en caso de necesitar recuperarlo

In [82]:
columnas_eliminar= ['price_period', 'rooms']

In [87]:
# C.2 Elimino columnas del df_analisis generando un nuevo df

df_filtrado_vf= df_filtrado.drop(columns=columnas_eliminar)

In [88]:
#Valido si se eliminaron las columnas (23 a 21)

df_filtrado_vf.shape

(10166, 21)

In [90]:
# C.3 Elimino filas donde los valores sean igual a 0
df_filtrado_vf=df_filtrado_vf.dropna(subset=['surface_total','surface_covered','price','currency'])

In [93]:
# Valido si se eliminaron las filas a través de shape (debe reducirse los 10166 propiedades) y valido que no hayan quedado nulos en esos 4 atributos
df_filtrado_vf.isnull().sum()


id                                0
start_date                        0
lat                               0
lon                               0
l1                                0
l2                                0
l3                                0
l4                                0
bedrooms                       1880
bathrooms                       233
surface_total                     0
surface_covered                   0
currency                          0
property_type                     0
operation_type                    0
price                             0
año_publicacion                   0
mes                               0
año                               0
mes&año                           0
Cotizacion_Promedio_Mensual       0
dtype: int64

In [94]:
df_filtrado_vf.shape

(7709, 21)

#C.4 Imputo el valor medio a bedrooms y bathrooms

El tratamiento de los nulos de los valores que reemplazo por la media quiero que consideren la media de propiedades con superficies similares, por lo cual realizaremos lo siguiente:
* Definimos intervalos de superficie cubierta (cada 50 m2)
* Asignamos a cada una de las propiedades un intervalo de acuerdo a su superficie cubierta 
* Calculamos la media de cada columna con nulos dentro del grupo de análisis.
* Imputamos los valores nulos usando la media calculada en el paso anterior

In [95]:
# 4) Defino intervalos
df_filtrado_vf['surface_covered_intervalo']= (df_filtrado_vf ['surface_covered']// 50) * 50

In [97]:
# Defino las columnas que voy a imputar los valores nulos con la media
columnas_imputar_media= ['bedrooms','bathrooms']

# Imputo valores nulos con la media del grupo correspondiente de acuerdo a los m2 de la propiedad.
for col in columnas_imputar_media:
    df_filtrado_vf[col] = df_filtrado_vf.groupby('surface_covered_intervalo')[col].transform(
        lambda x: x.fillna(x.mean())
    )

In [98]:
#Verifico que no haya nulos en ninguna columna

df_filtrado_vf.isnull().sum()

id                             0
start_date                     0
lat                            0
lon                            0
l1                             0
l2                             0
l3                             0
l4                             0
bedrooms                       2
bathrooms                      1
surface_total                  0
surface_covered                0
currency                       0
property_type                  0
operation_type                 0
price                          0
año_publicacion                0
mes                            0
año                            0
mes&año                        0
Cotizacion_Promedio_Mensual    0
surface_covered_intervalo      0
dtype: int64

In [99]:
# Para ver como quedo nuestro dataset voy a utilizar describe(), sentencia que me va a permitir indagar los valores numéricos de mi dataset

df_filtrado_vf.describe()

,id,start_date,lat,lon,bedrooms,bathrooms,surface_total,surface_covered,price,año_publicacion,mes,año,Cotizacion_Promedio_Mensual,surface_covered_intervalo
count,7709.000000,7709,7709.000000,7709.000000,7707.000000,7708.000000,7709.000000,7709.00000,7709.000000,7709.000000,7709.000000,7709.000000,7709.000000,7709.000000
mean,438893.371384,2019-12-26 04:14:47.274614016,-39.910210,-23.422168,3.348575,3.077263,544.688805,278.93073,48104.859385,2019.447269,6.994163,2019.447269,59.801939,259.534311
min,271.000000,2019-07-04 00:00:00,-59.030171,-34.592448,1.000000,1.000000,20.000000,1.00000,100.000000,2019.000000,1.000000,2019.000000,42.580000,0.000000
25%,178919.000000,2019-09-12 00:00:00,-58.700615,-34.446869,3.000000,2.000000,200.000000,160.00000,9800.000000,2019.000000,5.000000,2019.000000,56.463333,150.000000
50%,423306.000000,2019-12-07 00:00:00,-58.614400,-34.403817,3.000000,3.000000,350.000000,220.00000,37000.000000,2019.000000,7.000000,2019.000000,59.875550,200.000000
75%,656591.000000,2020-04-18 00:00:00,0.000000,0.000000,4.000000,4.000000,758.000000,309.00000,65000.000000,2020.000000,10.000000,2020.000000,65.763850,300.000000
max,999978.000000,2020-07-27 00:00:00,0.000000,0.000000,36.000000,10.000000,169000.000000,169000.00000,650000.000000,2020.000000,12.000000,2020.000000,71.479524,169000.000000
std,289028.563583,NaN,27.372948,16.064363,0.912996,1.310170,2105.452666,2004.74938,51524.656790,0.497244,3.144850,0.497244,7.325064,2005.232330


## 3. ANÁLISIS EXPLOTATORIO DATASET FINAL

Vamos a realizar un análisis explotatorio a través de herramientas de visualización.
Utilizaremos la bibilioteca plotly para generar datos interactivos que nos permitan dimensionar los datos que tenemos en nuestro dataset

In [104]:
# instalo plotly para poder utilizarlo
!pip install plotly

   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   -------- ------------------------------- 2.1/9.8 MB 12.5 MB/s eta 0:00:01
   -------------------------- ------------- 6.6/9.8 MB 17.1 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.8 MB 18.3 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 14.0 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   -------------------- ------------------- 1/2 [plotly]
   -------------------- ------------------- 1/2 [plotly]
   -------------------- ------------------- 1/2 [plotly]
   -------------------- ------------------- 1/2 [plotly]
   -------------------- ------------------- 1/2 [plotly]
   -------------------- ------------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [114]:
# Importo plotly

import plotly.express as px

In [ ]:
# Agrupar por barrio y contar cantidad de propiedades
conteo = df_filtrado_vf['l3'].value_counts().reset_index()
conteo.columns = ['Barrio', 'Cantidad']

# Crear gráfico interactivo
fig = px.bar(conteo, x='Barrio', y='Cantidad', title='Cantidad de propiedades por barrio')



In [120]:
#Como se me genera un error para mostrarlo en la notebook de Jupyter le indico que me lo muestre en el navegador cuando lo ejecuto
import plotly.io as pio
pio.renderers.default = 'browser'

fig.show()


In [130]:
# Voy a realizar un gráfico para calcular el precio por m² por barrio

#Calculo el precio por m2
df_filtrado_vf['precio_m2'] = df_filtrado_vf['price'] / df_filtrado_vf['surface_total']

# Agrupar por barrio y calcular promedio
promedio_m2 = df_filtrado_vf.groupby('l3')['precio_m2'].mean().reset_index()
promedio_m2.columns = ['Barrio', 'Precio promedio m²']

# Ordenar de mayor a menor
promedio_m2 = promedio_m2.sort_values(by='Precio promedio m²', ascending=False)

# Crear gráfico con etiquetas
fig2 = px.bar(
    promedio_m2.round(2),
    x='Barrio',
    y='Precio promedio m²',
    text='Precio promedio m²',
    title='Precio promedio por m² por barrio (ordenado de mayor a menor)'
)
fig2.update_traces(textposition='outside')
fig2.show()


En las visualizaciones observo que me quedaron propiedades con el l3 "0". Esto es producto de un error de decisión respecto al tratamiento de nulos. 
En este caso decidi imputarlo con un valor constante, cuando este dato es fundamental para el análisis que queremos efectuar. 
Sin contar con la geolocalización del l3 no voy a poder identificar a que barrio pertenece la propiedad por lo que la decisión correcta debería haber sido eliminar las filas.
Vamos a efectuarlo en esta instancia y vamos a volver a correr los gráficos efectuados previamente. 

In [131]:
df_filtrado_vf2 = df_filtrado_vf[df_filtrado_vf['l3'] != 0]


In [135]:
df_filtrado_vf2.shape

(7637, 23)

In [137]:
df_filtrado_vf2.head(1)

,id,start_date,lat,lon,l1,l2,l3,l4,bedrooms,bathrooms,surface_total,surface_covered,currency,property_type,operation_type,price,año_publicacion,mes,año,mes&año,Cotizacion_Promedio_Mensual,surface_covered_intervalo,precio_m2
5921,271,2019-07-07,-58.649171,-34.397301,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,1.0,2.0,72.0,59.0,ARS,Casa,Alquiler,28000.0,2019,7,2019,7&2019,42.58,50.0,388.888889


Cuando realizón un análisis final de mi dataset veo que me quedaron dos columnas con la misma información: "año publicación" y "año". Voy a eliminar una del nuevo dataset generado

In [139]:
df_filtrado_vf2=df_filtrado_vf2.drop(['año_publicacion'],axis=1)

In [141]:
#Vuelvo a realizar los gráficos realizados anteriormente con el nuevo df

# Agrupar por barrio y contar cantidad de propiedades
conteo = df_filtrado_vf2['l3'].value_counts().reset_index()
conteo.columns = ['Barrio', 'Cantidad']

# Crear gráfico interactivo
fig = px.bar(conteo, x='Barrio', y='Cantidad', title='Cantidad de propiedades por barrio')

fig.show()

In [143]:
# Valor por m2 por barrio con nuevo df

#Calculo el precio por m2
df_filtrado_vf2['precio_m2'] = df_filtrado_vf2['price'] / df_filtrado_vf2['surface_total']

# Agrupar por barrio y calcular promedio
promedio_m2 = df_filtrado_vf2.groupby('l3')['precio_m2'].mean().reset_index()
promedio_m2.columns = ['Barrio', 'Precio promedio m²']

# Ordenar de mayor a menor
promedio_m2 = promedio_m2.sort_values(by='Precio promedio m²', ascending=False)

# Crear gráfico con etiquetas
fig2 = px.bar(
    promedio_m2.round(2),
    x='Barrio',
    y='Precio promedio m²',
    text='Precio promedio m²',
    title='Precio promedio por m² por barrio (ordenado de mayor a menor)'
)
fig2.update_traces(textposition='outside')
fig2.show()


Con los gráficos efectuados comenzamos a contar con insights a partir de visualizaciones que nos permitieron explorar los datos y comenzar a efectuar un análisis que va a ser vital para la confirmación de la hipotésis 2 donde buscaremos realizar una comparativa entre los precios de alquiler vs la cuota de un crédito hipotecario en base a los valores de venta de las propiedades analizadas en el curso de data science I. 